In [1]:
import math
import time
import multiprocessing

from SALib.sample import saltelli
from SALib.analyze import sobol
from numba import njit
import numpy as np

In [6]:
a = 2
b = 0.1

In [7]:
problem = {
    "num_vars": 4,
    "names": ["x1", "x2", "x3", "x4"],
    'bounds': [
        [-math.pi, math.pi],
        [-math.pi, math.pi],
        [-math.pi, math.pi],
        [-math.pi, math.pi],
    ]
}

In [12]:
def timer(func):
    def wrapper(*args):
        print(f"Start {func.__name__}")
        start = time.time()
        response = func(*args)
        print(f"func.__name__ completed via {time.time() - start} sec.\n")
        return response
    return wrapper

In [13]:
def multidimensional_scalar_function(x):
    return x[0] ** a + math.exp(x[1]) + b * x[2] * math.sin(x[3])

In [14]:
@timer
def generate_samples(n):
    param_values = saltelli.sample(problem, n)
    return param_values

In [15]:
@timer
def generate_scalars(param_values):
    responses = np.zeros(param_values.shape[0])
    for idx, x, in enumerate(param_values):
        responses[idx] = multidimensional_scalar_function(x)
    return responses

### Обычный режим работы

In [16]:
n = 1000000
responses = generate_samples(n)
scalars = generate_scalars(responses)
print(f"Start analyze by Sobol method")
start = time.time()
Si = sobol.analyze(problem, scalars, print_to_console=True)
print(f"Sobol spent {time.time() - start}")
print('First-order Indices')
print(Si['S1'])
print('Total Indices for each variable')
print(Si['ST'])
print('Second-order Indices')
print(Si['S2'])

Start generate_samples


<ipython-input-14-a3c3c7e8e3b9>:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
C:\Users\Egor_Pilat\Anaconda3\lib\site-packages\SALib\sample\saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1000000) is equal to `2^n`.
        
  warnings.warn(msg)


func.__name__ completed via 63.156038999557495 sec.

Start generate_scalars
func.__name__ completed via 16.29341197013855 sec.

Start analyze by Sobol method
          ST   ST_conf
x1  0.229218  0.000655
x2  0.770349  0.002010
x3  0.000435  0.000002
x4  0.000435  0.000002
              S1   S1_conf
x1  2.292226e-01  0.001254
x2  7.703492e-01  0.002994
x3 -4.440148e-07  0.000057
x4  4.625675e-08  0.000055
                S2   S2_conf
(x1, x2) -0.000008  0.002284
(x1, x3) -0.000005  0.001760
(x1, x4) -0.000005  0.001757
(x2, x3)  0.000002  0.004805
(x2, x4)  0.000002  0.004808
(x3, x4)  0.000436  0.000075
Sobol spent 196.41722559928894
First-order Indices
[ 2.29222587e-01  7.70349235e-01 -4.44014846e-07  4.62567455e-08]
Total Indices for each variable
[2.29218438e-01 7.70348610e-01 4.35464980e-04 4.35463593e-04]
Second-order Indices
[[            nan -7.98788276e-06 -4.89601633e-06 -5.35835155e-06]
 [            nan             nan  1.84800406e-06  1.94148559e-06]
 [            nan      

###  Добавим njit

In [19]:
@njit
def multidimensional_scalar_function(x):
    return x[0] ** a + math.exp(x[1]) + b * x[2] * math.sin(x[3])

n = 100
responses = generate_samples(n)
scalars = generate_scalars(responses)
print(f"Start analyze by Sobol method")
start = time.time()
Si = sobol.analyze(problem, scalars, print_to_console=True)
print(f"Sobol spent {time.time() - start}")
print('First-order Indices')
print(Si['S1'])
print('Total Indices for each variable')
print(Si['ST'])
print('Second-order Indices')
print(Si['S2'])

Start generate_samples
func.__name__ completed via 0.015000581741333008 sec.

Start generate_scalars


<ipython-input-14-a3c3c7e8e3b9>:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
C:\Users\Egor_Pilat\Anaconda3\lib\site-packages\SALib\sample\saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (100) is equal to `2^n`.
        
  warnings.warn(msg)


func.__name__ completed via 0.40579724311828613 sec.

Start analyze by Sobol method
          ST   ST_conf
x1  0.228890  0.078932
x2  0.835307  0.170470
x3  0.000450  0.000179
x4  0.000419  0.000189
          S1   S1_conf
x1  0.218051  0.129216
x2  0.832487  0.319353
x3  0.002395  0.007011
x4  0.005369  0.007246
                S2   S2_conf
(x1, x2)  0.024489  0.258654
(x1, x3) -0.047381  0.170341
(x1, x4) -0.047007  0.170697
(x2, x3) -0.063187  0.480767
(x2, x4) -0.065330  0.482084
(x3, x4) -0.001129  0.007796
Sobol spent 0.02580571174621582
First-order Indices
[0.21805132 0.83248745 0.00239522 0.00536945]
Total Indices for each variable
[2.28889555e-01 8.35307430e-01 4.50286184e-04 4.19354947e-04]
Second-order Indices
[[        nan  0.02448924 -0.0473809  -0.04700736]
 [        nan         nan -0.06318664 -0.06533041]
 [        nan         nan         nan -0.00112892]
 [        nan         nan         nan         nan]]


In [20]:
@njit
def multidimensional_scalar_function(x):
    return x[0] ** a + math.exp(x[1]) + b * x[2] * math.sin(x[3])

n = 10000
responses = generate_samples(n)
scalars = generate_scalars(responses)
print(f"Start analyze by Sobol method")
start = time.time()
Si = sobol.analyze(problem, scalars, print_to_console=True)
print(f"Sobol spent {time.time() - start}")
print('First-order Indices')
print(Si['S1'])
print('Total Indices for each variable')
print(Si['ST'])
print('Second-order Indices')
print(Si['S2'])

Start generate_samples


<ipython-input-14-a3c3c7e8e3b9>:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
C:\Users\Egor_Pilat\Anaconda3\lib\site-packages\SALib\sample\saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (10000) is equal to `2^n`.
        
  warnings.warn(msg)


func.__name__ completed via 0.7631440162658691 sec.

Start generate_scalars
func.__name__ completed via 0.12463068962097168 sec.

Start analyze by Sobol method
          ST   ST_conf
x1  0.229290  0.008034
x2  0.769918  0.017781
x3  0.000435  0.000017
x4  0.000436  0.000018
          S1   S1_conf
x1  0.229564  0.013462
x2  0.769990  0.029873
x3  0.000003  0.000563
x4  0.000114  0.000622
                S2   S2_conf
(x1, x2) -0.000543  0.024435
(x1, x3) -0.000609  0.018423
(x1, x4) -0.000722  0.018406
(x2, x3) -0.000256  0.046920
(x2, x4) -0.000240  0.046945
(x3, x4)  0.000444  0.000816
Sobol spent 0.8505649566650391
First-order Indices
[2.29564390e-01 7.69990137e-01 3.36169484e-06 1.13617391e-04]
Total Indices for each variable
[2.29289754e-01 7.69917628e-01 4.35334625e-04 4.35666456e-04]
Second-order Indices
[[        nan -0.00054313 -0.00060852 -0.00072163]
 [        nan         nan -0.00025557 -0.00024034]
 [        nan         nan         nan  0.00044356]
 [        nan         nan 

In [17]:
@njit
def multidimensional_scalar_function(x):
    return x[0] ** a + math.exp(x[1]) + b * x[2] * math.sin(x[3])

n = 1000000
responses = generate_samples(n)
scalars = generate_scalars(responses)
print(f"Start analyze by Sobol method")
start = time.time()
Si = sobol.analyze(problem, scalars, print_to_console=True)
print(f"Sobol spent {time.time() - start}")
print('First-order Indices')
print(Si['S1'])
print('Total Indices for each variable')
print(Si['ST'])
print('Second-order Indices')
print(Si['S2'])

Start generate_samples


<ipython-input-14-a3c3c7e8e3b9>:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
C:\Users\Egor_Pilat\Anaconda3\lib\site-packages\SALib\sample\saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1000000) is equal to `2^n`.
        
  warnings.warn(msg)


func.__name__ completed via 67.6929099559784 sec.

Start generate_scalars
func.__name__ completed via 5.984112501144409 sec.

Start analyze by Sobol method
          ST   ST_conf
x1  0.229218  0.000740
x2  0.770349  0.001945
x3  0.000435  0.000002
x4  0.000435  0.000002
              S1   S1_conf
x1  2.292226e-01  0.001545
x2  7.703492e-01  0.002649
x3 -4.440148e-07  0.000059
x4  4.625675e-08  0.000055
                S2   S2_conf
(x1, x2) -0.000008  0.002662
(x1, x3) -0.000005  0.001958
(x1, x4) -0.000005  0.001959
(x2, x3)  0.000002  0.004720
(x2, x4)  0.000002  0.004713
(x3, x4)  0.000436  0.000078
Sobol spent 194.91741228103638
First-order Indices
[ 2.29222587e-01  7.70349235e-01 -4.44014846e-07  4.62567455e-08]
Total Indices for each variable
[2.29218438e-01 7.70348610e-01 4.35464980e-04 4.35463593e-04]
Second-order Indices
[[            nan -7.98788276e-06 -4.89601633e-06 -5.35835155e-06]
 [            nan             nan  1.84800406e-06  1.94148559e-06]
 [            nan        

### Добавим параметр для распараллеливания внутри SALib

In [18]:
responses = generate_samples(n)
scalars = generate_scalars(responses)
print(f"Start analyze by Sobol method")
start = time.time()
Si = sobol.analyze(problem, scalars, print_to_console=True, parallel=True, n_processors=4)
print(f"Sobol spent {time.time() - start}")
print('First-order Indices')
print(Si['S1'])
print('Total Indices for each variable')
print(Si['ST'])
print('Second-order Indices')
print(Si['S2'])

Start generate_samples


<ipython-input-14-a3c3c7e8e3b9>:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
C:\Users\Egor_Pilat\Anaconda3\lib\site-packages\SALib\sample\saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1000000) is equal to `2^n`.
        
  warnings.warn(msg)


func.__name__ completed via 61.56002449989319 sec.

Start generate_scalars
func.__name__ completed via 5.127791881561279 sec.

Start analyze by Sobol method
          ST   ST_conf
x1  0.229218  0.000850
x2  0.770349  0.001644
x3  0.000435  0.000002
x4  0.000435  0.000002
              S1   S1_conf
x1  2.292226e-01  0.001157
x2  7.703492e-01  0.002816
x3 -4.440148e-07  0.000059
x4  4.625675e-08  0.000057
                S2   S2_conf
(x1, x2) -0.000008  0.002336
(x1, x3) -0.000005  0.001532
(x1, x4) -0.000005  0.001533
(x2, x3)  0.000002  0.004476
(x2, x4)  0.000002  0.004480
(x3, x4)  0.000436  0.000086
Sobol spent 150.3744809627533
First-order Indices
[ 2.29222587e-01  7.70349235e-01 -4.44014846e-07  4.62567455e-08]
Total Indices for each variable
[2.29218438e-01 7.70348610e-01 4.35464980e-04 4.35463593e-04]
Second-order Indices
[[            nan -7.98788276e-06 -4.89601633e-06 -5.35835155e-06]
 [            nan             nan  1.84800406e-06  1.94148559e-06]
 [            nan        

#### Наиболее важные параметры x1, x2 